## Using (most) of the chip to encode the 3 qubit triangle

In [1]:
# Handle the relevant import statements before moving on...
import sys
sys.path.append("../dwaveutils/dwavetools")
sys.path.append("../dwaveutils/probrep")
from dictrep import DictRep
from dwave.system.samplers import DWaveSampler
from dwavetools import (find_heff_s, make_dwave_schedule, create_heff_csv,
                        nqubit_1pauli, nqubit_2pauli, loadAandB,
                        make_numeric_schedule, time_interpolation,
                        get_numeric_H, densify_unitcell)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import qutip as qt
from operator import add

Load in chip data and get working qubits/couplers. 
Also, "block_H" refers to the specific qubits that connect the "blocks" that make up the massive "1 qubit chains." 

In [2]:
chip = DWaveSampler()
wqubits = chip.properties['qubits']
wcouplers = chip.properties['couplers']
block_H = {(44, 52): 'J2', (1243, 1371): 'J1', (1168, 1296): 'J1'}

Here, we will form the large "blocks" that densley connect a bunch of qubits in a rectangle that represents a single qubit.

In [50]:
def dense_rectangle_qubit(fqbit, width, height, hweight, Jweight):
    '''
    Creates a width x height densely connected qubit chain starting with fqbit, giving
    each qubit along the chain a linear weight of hweight and coupling stregnth of Jweight. 
    '''
    q_H = {}
    
    ucell = 1
    while ucell <= (width*height+1):
        print(ucell)
        print(fqbit)
        # if not at the edge of rectangle, go to next unit cell to the right
        # and make couplings that connect these unit cells
        if ucell % width != 0: 
            couplings = [[fqbit+4+i, fqbit+12+i] for i in range(4)]
            fqbit += 8
        # otherwise, go to next row and connect edge unit cell with those BELOW
        # unless you are actually on the final row, of course; then don't connect down
        elif (ucell % width == 0) and (ucell < width*height+1):
            couplings = [[fqbit+i, fqbit+128+i] for i in range(4)]
            fqbit += 88

        uc_H = densify_unitcell(fqbit, hweight, Jweight, wqubits, wcouplers)
        q_H.update(uc_H)
        count = 0
        for coupling in couplings:
            if coupling in wcouplers:
                q_H.update({tuple(coupling): Jweight})
                count += 1         
        if count == 0:
            return("Chain is broken; no connections availible.")
            break

        ucell += 1
        
    return q_H

In [55]:
# create the first qubit by starting with qubit 0 and creating a dense 6 x 10 block
#first_qubit = dense_rectangle_qubit(0, 6, 10, 1, 1)

In [56]:
# create the second qubit by starting with qubit 48 and creating another dense 6 x 10 block
#second_qubit = dense_rectangle_qubit(48, 6, 10, 1, 1)

In [57]:
# create the final qubit by starting with qubit 1280 and creating a dense 10 x 6 block
#third_qubit = dense_rectangle_qubit(1280, 10, 6, 1, 1)